In [12]:
from google.cloud import storage
from google.cloud import bigquery
from google.oauth2 import service_account
from sklearn.linear_model import LogisticRegression
import db_dtypes
import pandas as pd
import os
import pickle5 as pickle
import numpy as np

In [7]:
# Get credentials.
gcp_cred = service_account.Credentials.from_service_account_file('../gcloud_private_key.json')

In [9]:
# Construct a BigQuery client object.
client = bigquery.Client(credentials=gcp_cred)

# Build query
query = """
    SELECT *
    FROM HW_DB.schedule_game_results
    WHERE gameType='R'
"""
query_job = client.query(query)  # Make an API request.

df = query_job.to_dataframe()
df.head()

,season,gameDate,gameID,gameType,homeScore,homeTeamID,awayScore,awayTeamID
0,20212022,2021-10-12,2021020001,R,2,14,6,5
1,20212022,2021-10-12,2021020002,R,4,54,3,55
2,20212022,2021-10-13,2021020003,R,2,10,1,8
3,20212022,2021-10-13,2021020004,R,5,15,1,3
4,20212022,2021-10-13,2021020015,R,4,24,1,52


In [ ]:

def create_lag(df, p, sortList, partList, lagList):
    """
    Cette fonction prend comme intrant un df, une liste de variables pour lesquelles il faut
    creer des valeurs retardees, le nombre de retard, les variables sur lesquelles il faut sort,
    les variables de partition et la liste de varibles pour lesquelles on veut des valeurs retardees.
    La fonction retourne un df augmente des variables retardees
    """
    df = df.sort_values(by=sortList)
    
    for feature in lagList:
        for i in range(p):
            df['{}.L{}'.format(feature,i+1)] = df.groupby(partList)[feature].shift(i+1)
    
    return df



In [14]:
df['home_win'] = np.where(df['homeScore'] > df['awayScore'], True, False)
df['ones'] = 1
lagged = create_lag(df=df,
                    p=1,
                    sortList=['season', 'g', 'gamePk'], 
                    partList=['teams.home.team.name', 'season'],
                    lagList=['teams.home.score','teams.away.score'])



,season,gameDate,gameID,gameType,homeScore,homeTeamID,awayScore,awayTeamID,home_win,ones
0,20212022,2021-10-12,2021020001,R,2,14,6,5,False,1
1,20212022,2021-10-12,2021020002,R,4,54,3,55,True,1
2,20212022,2021-10-13,2021020003,R,2,10,1,8,True,1
3,20212022,2021-10-13,2021020004,R,5,15,1,3,True,1
4,20212022,2021-10-13,2021020015,R,4,24,1,52,True,1


In [15]:
y = df['home_win']
X = df['ones']
lr_model = LogisticRegression(random_state=0).fit(X, y)

ValueError: Expected 2D array, got 1D array instead:
array=[1. 1. 1. ... 1. 1. 1.].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.